In [ ]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

path = r'C:\Users\user\Documents\GitHub\Notebooks-on-ml\TITANIC\Data\train.csv'
df = pd.read_csv(path)

train_df_raw, val_df_raw = train_test_split(df, test_size=0.2, random_state=42)

def get_safe_survival_mapping(train_fs, test_fs):
    train_fs = train_fs.copy()
    test_fs = test_fs.copy()

    train_fs['Surname'] = train_fs['Name'].apply(lambda x: x.split(',')[0].strip())
    test_fs['Surname'] = test_fs['Name'].apply(lambda x: x.split(',')[0].strip())

    train_fs['Family_Survival'] = 0.5
    test_fs['Family_Survival'] = 0.5

    for _, group in train_fs.groupby(['Surname', 'Fare']):
        if len(group) > 1:
            for ind, row in group.iterrows():
                others = group.drop(ind)
                if others['Survived'].max() == 1.0:
                    train_fs.loc[ind, 'Family_Survival'] = 1
                elif others['Survived'].max() == 0.0:
                    train_fs.loc[ind, 'Family_Survival'] = 0

    for _, group in train_fs.groupby('Ticket'):
        if len(group) > 1:
            for ind, row in group.iterrows():
                if train_fs.loc[ind, 'Family_Survival'] == 0.5:
                    others = group.drop(ind)
                    if others['Survived'].max() == 1.0:
                        train_fs.loc[ind, 'Family_Survival'] = 1
                    elif others['Survived'].max() == 0.0:
                        train_fs.loc[ind, 'Family_Survival'] = 0

    for ind, row in test_fs.iterrows():
        fam_in_train = train_fs[(train_fs['Surname'] == row['Surname']) & (train_fs['Fare'] == row['Fare'])]
        ticket_in_train = train_fs[train_fs['Ticket'] == row['Ticket']]

        combined = pd.concat([fam_in_train, ticket_in_train])

        if len(combined) > 0:
            if combined['Survived'].max() == 1.0:
                test_fs.loc[ind, 'Family_Survival'] = 1
            elif combined['Survived'].max() == 0.0:
                test_fs.loc[ind, 'Family_Survival'] = 0

    full_mapping = pd.concat([train_fs[['PassengerId', 'Family_Survival']],
                                    test_fs[['PassengerId', 'Family_Survival']]])
    return full_mapping

def prepare_data(df_input, survival_mapping):
    df = df_input.copy()
    
    df['Ticket_Group_Size'] = df.groupby('Ticket')['Ticket'].transform('count')
    df = df.merge(survival_mapping, on='PassengerId', how='left')
    df['Title'] = df['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
    df['Title'] = df['Title'].replace(['Rev', 'Dr'], 'Service')
    df['Title'] = df['Title'].replace(['Jonkheer', 'Don', 'Sir', 'Lady', 'Countess', 'Dona'], 'Noble')
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major'], 'Officer')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(float)
    df['Has_Cabin'] = df['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
    df['Deck'] = df['Cabin'].str[0].fillna('M')
    df['Deck'] = df['Deck'].replace(['A', 'B', 'C'], 'Top')
    df['Deck'] = df['Deck'].replace(['D', 'E'], 'Middle')
    df['Deck'] = df['Deck'].replace(['F', 'G', 'T', 'M'], 'Low')
    df['Age'] = df.groupby('Title')['Age'].transform(lambda x: x.fillna(x.median()))
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    
    cols_to_drop = ['Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch']
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
    
    df = pd.get_dummies(df, columns=['Title', 'Deck', 'Embarked'])
    
    return df.astype(float)

fs_map = get_safe_survival_mapping(train_df_raw, val_df_raw)

X_train = prepare_data(train_df_raw, fs_map).drop(columns=['Survived'])
y_train = train_df_raw['Survived']

X_val = prepare_data(val_df_raw, fs_map).drop(columns=['Survived'])
y_val = val_df_raw['Survived']

X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

def objective(trial):
    xgb_params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }
    w1 = trial.suggest_float('weight_rf', 0.1, 1.0)
    w2 = trial.suggest_float('weight_xgb', 0.1, 1.0)
    
    rf_model = RandomForestClassifier(
        n_estimators=370, max_depth=6, min_samples_split=12, 
        min_samples_leaf=4, random_state=42
    )
    
    xgb_model = XGBClassifier(**xgb_params)
    
    ensemble = VotingClassifier(
        estimators=[('rf', rf_model), ('xgb', xgb_model)],
        voting='soft'
    )
    
    score = cross_val_score(ensemble, X_train, y_train, cv=5).mean()
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

best_xgb = XGBClassifier(**study.best_params, random_state=42)
best_rf = RandomForestClassifier(n_estimators=370, max_depth=6, min_samples_split=12, min_samples_leaf=4, random_state=42)

calibrated_xgb = CalibratedClassifierCV(best_xgb, method='sigmoid', cv=5)
calibrated_rf = CalibratedClassifierCV(best_rf, method='sigmoid', cv=5)

final_model = VotingClassifier(
    estimators=[('rf', calibrated_rf), ('xgb', calibrated_xgb)],
    voting='soft',
    weights=[study.best_params['weight_rf'], study.best_params['weight_xgb']]
)
final_model.fit(X_train, y_train)
print("-" * 30)
print(f"Лучший CV результат: {study.best_value:.2%}")
print(f"Честная точность на валидации: {final_model.score(X_val, y_val):.2%}")
print("best_params = {")
for key, value in study.best_params.items():
    if isinstance(value, str):
        print(f"    '{key}': '{value}',")
    else:
        print(f"    '{key}': {value},")
print("    'random_state': 42")
print("}") 
print("-" * 30)

[I 2025-12-29 14:19:14,429] A new study created in memory with name: no-name-aa53f85e-0eba-4155-9d79-8557b09be240
d:\system\python\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:19:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\system\python\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:19:15] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\system\python\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:19:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\system\python\Lib\site-packages\xgboost\training.py:199: UserWarning: [14:19:16] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\le

------------------------------
Лучший CV результат: 85.39%
Честная точность на валидации: 85.47%
best_params = {
    'n_estimators': 284,
    'max_depth': 8,
    'learning_rate': 0.010129089335270519,
    'subsample': 0.7450299265656695,
    'colsample_bytree': 0.7041909842845945,
    'weight_rf': 0.26644968728053153,
    'weight_xgb': 0.880537546649935,
    'random_state': 42
}
------------------------------
Честная точность на валидации: 85.47%
best_params = {
    'n_estimators': 284,
    'max_depth': 8,
    'learning_rate': 0.010129089335270519,
    'subsample': 0.7450299265656695,
    'colsample_bytree': 0.7041909842845945,
    'weight_rf': 0.26644968728053153,
    'weight_xgb': 0.880537546649935,
    'random_state': 42
}
------------------------------


In [12]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

importances = pd.Series(final_model.named_estimators_['rf'].feature_importances_, index=X_train.columns)
print(importances.sort_values(ascending=True).head(40))
importances.nlargest(10).plot(kind='barh')
plt.title('Топ-10 важных признаков по мнению Random Forest')
plt.show()

errors_df = val_df_raw.copy()
errors_df['Actual'] = val_df_raw['Survived']
errors_df['Predicted'] = final_model.predict(X_val)
errors = errors_df[errors_df['Actual'] != errors_df['Predicted']]
errors.head(10)

y_probs = final_model.predict_proba(X_val)[:, 1]
y_pred_custom = (y_probs >= 0.5).astype(int)
sns.heatmap(confusion_matrix(y_val, y_pred_custom), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')

AttributeError: 'CalibratedClassifierCV' object has no attribute 'feature_importances_'